In [1]:
import warnings
warnings.filterwarnings('ignore')

from typing import List, Dict
import autogen
import torch

from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

import json


print("🚀 Loading models...\n")

# Create a Hugging Face text2text pipeline
pipe = pipeline(
    task="text2text-generation",
    model="google/flan-t5-base",
    max_length=200,
    temperature=0.7
)

# Wrap it in a LangChain-compatible LLM
llm = HuggingFacePipeline(pipeline=pipe)

print("✓ Models loaded!\n")

# Example usage
prompt = "Summarize this text: LangChain makes it easy to build LLM applications."
result = llm.invoke(prompt)
print(result)

🚀 Loading models...



Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/var/folders/7p/3ckszphj4s920bw9vwb7wpdw0000gn/T/ipykernel_88282/729718009.py:28: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


✓ Models loaded!

LangChain is a software that simplifies the development of LLM applications.


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline


# Create a Hugging Face LLM pipeline
hf_pipe = pipeline("text2text-generation", model="google/flan-t5-base")
llm = HuggingFacePipeline(pipeline=hf_pipe)


def demo_langchain_basics():
    print("=" * 70)
    print("DEMO 1: LangChain - Intelligent Prompt Chains")
    print("=" * 70 + "\n")

    prompt = PromptTemplate(
        input_variables=["task"],
        template="Task: {task}\n\nProvide a detailed step-by-step solution:"
    )

    # Use new Runnable API
    chain = prompt | llm | StrOutputParser()

    task = "Create a Python function to calculate Fibonacci sequence"
    print(f"Task: {task}\n")

    # Use invoke() instead of run()
    result = chain.invoke({"task": task})

    print(f"LangChain Response:\n{result}\n")
    print("✓ LangChain demo complete\n")


def demo_langchain_multi_step():
    print("=" * 70)
    print("DEMO 2: LangChain - Multi-Step Reasoning")
    print("=" * 70 + "\n")

    planner = PromptTemplate(
        input_variables=["goal"],
        template="Break down this goal into 3 steps: {goal}"
    )
    executor = PromptTemplate(
        input_variables=["step"],
        template="Explain how to execute this step: {step}"
    )

    # New Runnable syntax for each stage
    plan_chain = planner | llm | StrOutputParser()
    exec_chain = executor | llm | StrOutputParser()

    goal = "Build a machine learning model"
    print(f"Goal: {goal}\n")

    # Again, use invoke() not run()
    plan = plan_chain.invoke({"goal": goal})
    print(f"Plan:\n{plan}\n")

    print("Executing first step...")
    execution = exec_chain.invoke({"step": "Collect and prepare data"})
    print(f"Execution:\n{execution}\n")
    print("✓ Multi-step reasoning complete\n")


# Run both demos
demo_langchain_basics()
demo_langchain_multi_step()


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 94ca1497-81a3-40fa-bb8d-0f797d647dc4)')' thrown while requesting HEAD https://huggingface.co/google/flan-t5-base/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Device set to use mps:0


DEMO 1: LangChain - Intelligent Prompt Chains

Task: Create a Python function to calculate Fibonacci sequence

LangChain Response:
The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The

✓ LangChain demo complete

DEMO 2: LangChain - M

In [3]:
class SimpleAgent:
   def __init__(self, name: str, role: str, llm_pipeline):
       self.name = name
       self.role = role
       self.pipe = llm_pipeline
       self.memory = []
   def process(self, message: str) -> str:
       prompt = f"You are a {self.role}.\nUser: {message}\nYour response:"
       response = self.pipe(prompt, max_length=150)[0]['generated_text']
       self.memory.append({"user": message, "agent": response})
       return response
   def __repr__(self):
       return f"Agent({self.name}, role={self.role})"


def demo_simple_agents():
   print("="*70)
   print("DEMO 3: Simple Multi-Agent System")
   print("="*70 + "\n")
   researcher = SimpleAgent("Researcher", "research specialist", pipe)
   coder = SimpleAgent("Coder", "Python developer", pipe)
   reviewer = SimpleAgent("Reviewer", "code reviewer", pipe)
   print("Agents created:", researcher, coder, reviewer, "\n")
   task = "Create a function to sort a list"
   print(f"Task: {task}\n")
   print(f"[{researcher.name}] Researching...")
   research = researcher.process(f"What's the best approach to: {task}")
   print(f"Research: {research[:100]}...\n")
   print(f"[{coder.name}] Coding...")
   code = coder.process(f"Write Python code to: {task}")
   print(f"Code: {code[:100]}...\n")
   print(f"[{reviewer.name}] Reviewing...")
   review = reviewer.process(f"Review this approach: {code[:50]}")
   print(f"Review: {review[:100]}...\n")
   print("✓ Multi-agent workflow complete\n")

In [4]:
def demo_autogen_conceptual():
   print("="*70)
   print("DEMO 4: AutoGen Concepts (Conceptual Demo)")
   print("="*70 + "\n")
   agent_config = {
       "agents": [
           {"name": "UserProxy", "type": "user_proxy", "role": "Coordinates tasks"},
           {"name": "Assistant", "type": "assistant", "role": "Solves problems"},
           {"name": "Executor", "type": "executor", "role": "Runs code"}
       ],
       "workflow": [
           "1. UserProxy receives task",
           "2. Assistant generates solution",
           "3. Executor tests solution",
           "4. Feedback loop until complete"
       ]
   }
   print(json.dumps(agent_config, indent=2))
   print("\n📝 AutoGen Key Features:")
   print("  • Automated agent chat conversations")
   print("  • Code execution capabilities")
   print("  • Human-in-the-loop support")
   print("  • Multi-agent collaboration")
   print("  • Tool/function calling\n")
   print("✓ AutoGen concepts explained\n")


class MockLLM:
   def __init__(self):
       self.responses = {
           "code": "def fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)",
           "explain": "This is a recursive implementation of the Fibonacci sequence.",
           "review": "The code is correct but could be optimized with memoization.",
           "default": "I understand. Let me help with that task."
       }
   def generate(self, prompt: str) -> str:
       prompt_lower = prompt.lower()
       if "code" in prompt_lower or "function" in prompt_lower:
           return self.responses["code"]
       elif "explain" in prompt_lower:
           return self.responses["explain"]
       elif "review" in prompt_lower:
           return self.responses["review"]
       return self.responses["default"]


def demo_autogen_with_mock():
   print("="*70)
   print("DEMO 5: AutoGen with Custom LLM Backend")
   print("="*70 + "\n")
   mock_llm = MockLLM()
   conversation = [
       ("User", "Create a fibonacci function"),
       ("CodeAgent", mock_llm.generate("write code for fibonacci")),
       ("ReviewAgent", mock_llm.generate("review this code")),
   ]
   print("Simulated AutoGen Multi-Agent Conversation:\n")
   for speaker, message in conversation:
       print(f"[{speaker}]")
       print(f"{message}\n")
   print("✓ AutoGen simulation complete\n")

In [12]:
def demo_hybrid_system():
    print("=" * 70)
    print("DEMO 6: Hybrid LangChain + Multi-Agent System")
    print("=" * 70 + "\n")

    reasoning_prompt = PromptTemplate(
        input_variables=["problem"],
        template="Analyze this problem: {problem}\nWhat are the key steps?"
    )

    # Updated: Runnable chain
    reasoning_chain = reasoning_prompt | llm | StrOutputParser()

    # Example SimpleAgent class usage
    planner = SimpleAgent("Planner", "strategic planner", pipe)
    executor = SimpleAgent("Executor", "task executor", pipe)

    problem = "Optimize a slow database query"
    print(f"Problem: {problem}\n")

    print("[LangChain] Analyzing problem...")

    # ✅ Use .invoke() instead of .run()
    analysis = reasoning_chain.invoke({"problem": problem})
    print(f"Analysis: {analysis[:120]}...\n")

    print(f"[{planner.name}] Creating plan...")
    plan = planner.process(f"Plan how to: {problem}")
    print(f"Plan: {plan[:120]}...\n")

    print(f"[{executor.name}] Executing...")
    result = executor.process("Execute: Add database indexes")
    print(f"Result: {result[:120]}...\n")

    print("✓ Hybrid system complete\n")



if __name__ == "__main__":
   print("="*70)
   print("🤖 ADVANCED AGENTIC AI TUTORIAL")
   print("AutoGen + LangChain + HuggingFace")
   print("="*70 + "\n")
   demo_langchain_basics()
   demo_langchain_multi_step()
   demo_simple_agents()
   demo_autogen_conceptual()
   demo_autogen_with_mock()
   demo_hybrid_system()
   print("="*70)
   print("🎉 TUTORIAL COMPLETE!")
   print("="*70)
   print("\n📚 What You Learned:")
   print("  ✓ LangChain prompt engineering and chains")
   print("  ✓ Multi-step reasoning with LangChain")
   print("  ✓ Building custom multi-agent systems")
   print("  ✓ AutoGen architecture and concepts")
   print("  ✓ Combining LangChain + agents")
   print("  ✓ Using HuggingFace models (no API needed!)")
   print("\n💡 Key Takeaway:")
   print("  You can build powerful agentic AI systems without expensive APIs!")
   print("  Combine LangChain's chains with multi-agent architectures for")
   print("  intelligent, autonomous AI systems.")
   print("="*70 + "\n")

🤖 ADVANCED AGENTIC AI TUTORIAL
AutoGen + LangChain + HuggingFace

DEMO 1: LangChain - Intelligent Prompt Chains

Task: Create a Python function to calculate Fibonacci sequence

LangChain Response:
The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonacci sequence. The Fibonacci sequence is a fibonac

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Execution:
Collect and prepare data.

✓ Multi-step reasoning complete

DEMO 3: Simple Multi-Agent System

Agents created: Agent(Researcher, role=research specialist) Agent(Coder, role=Python developer) Agent(Reviewer, role=code reviewer) 

Task: Create a function to sort a list

[Researcher] Researching...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Research: To sort a list of items, click on the item you want to sort and then click on the item you want to s...

[Coder] Coding...


...

Code: a=list(map(int,input().split())) b=list(map(int,input().split())) c=list(map(int,input().split())) d...

[Reviewer] Reviewing...
Review: a=list(map(int,input().split())))...

✓ Multi-agent workflow complete

DEMO 4: AutoGen Concepts (Conceptual Demo)

{
  "agents": [
    {
      "name": "UserProxy",
      "type": "user_proxy",
      "role": "Coordinates tasks"
    },
    {
      "name": "Assistant",
      "type": "assistant",
      "role": "Solves problems"
    },
    {
      "name": "Executor",
      "type": "executor",
      "role": "Runs code"
    }
  ],
  "workflow": [
    "1. UserProxy receives task",
    "2. Assistant generates solution",
    "3. Executor tests solution",
    "4. Feedback loop until complete"
  ]
}

📝 AutoGen Key Features:
  • Automated agent chat conversations
  • Code execution capabilities
  • Human-in-the-loop support
  • Multi-agent collaboration
  • Tool/function calling

✓ AutoGen concepts explained

DEMO 5: AutoGen with Custom LLM Backend

Simulated

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Analysis: The first step is to optimize the database query. The second step is to optimize the database query. The third step is t...

[Planner] Creating plan...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Plan: Optimize a slow database query...

[Executor] Executing...
Result: Add database indexes...

✓ Hybrid system complete

🎉 TUTORIAL COMPLETE!

📚 What You Learned:
  ✓ LangChain prompt engineering and chains
  ✓ Multi-step reasoning with LangChain
  ✓ Building custom multi-agent systems
  ✓ AutoGen architecture and concepts
  ✓ Combining LangChain + agents
  ✓ Using HuggingFace models (no API needed!)

💡 Key Takeaway:
  You can build powerful agentic AI systems without expensive APIs!
  Combine LangChain's chains with multi-agent architectures for
  intelligent, autonomous AI systems.

